In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import math

from ipywidgets import interact

from ipywidgets import widgets
from tqdm.auto import tqdm

from dataclasses import dataclass
from typing import Union, Callable, Optional
from copy import deepcopy
from copy import error
from scipy.optimize import root_scalar, brentq
from dataclasses import dataclass
from scipy.special import iv
from scipy.stats import bernoulli
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import newton
from scipy.stats import norm

import warnings
from scipy.stats import norm
warnings.filterwarnings("ignore")

from vol.vol import Heston

import hestonmc
from importlib import reload
reload(hestonmc)

from hestonmc import MarketState, HestonParameters, mc_price, simulate_heston_euler, simulate_heston_andersen_qe, simulate_heston_andersen_tg

## Tests

In [24]:
heston_parameters = HestonParameters(kappa = 1.3125, gamma = 0.5125, rho = -0.3937, vbar = 0.0641, v0 = 0.3)
# kappa = 1.3125, gamma = 0.7125, rho = -0.3937, vbar = 0.0641, v0 = 0.3
state = MarketState(stock_price = 100., interest_rate = 0.2)

params = heston_parameters
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)
kwargs = {}

In [25]:
def get_payoff(maturity: float,
               strike: float,
               interest_rate: float = 0.):
    def payoff(St: np.ndarray):
        DF = np.exp( - interest_rate * maturity)
        return np.maximum(St - strike, 0.)*DF

    return payoff

### At the money

In [26]:
strike = 100.
T = 1.
payoff = get_payoff(T, strike, state.interest_rate)

In [27]:
theory = model.call_price(T, strike)
print(theory)

22.735939055676866


In [28]:
np.random.seed(42)
euler    = mc_price(N_T = 100, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_euler,
                    debug = True, params = params, T = T, **kwargs)
print(euler)

Number of iterations:   744
Number of simulations:  7440000
Absolute error:         0.04999268822764575
Confidence level:       0.05

26.29926158466383


In [29]:
np.random.seed(42)
andersen = mc_price(N_T = 100, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_qe, 
                    debug = True, params = params, T = T, **kwargs)
print(andersen)

Number of iterations:   739
Number of simulations:  7390000
Absolute error:         0.049995412365741544
Confidence level:       0.05

26.34789538008171


In [30]:
r_x = np.load(r"Data/anderson tg/r_x.npy")
f_nu_y = np.load(r"Data/anderson tg/f_nu_y.npy")
f_sigma_y = np.load(r"Data/anderson tg/f_sigma_y.npy")

In [31]:
np.random.seed(42)
kwargs = {'x_grid' : r_x, 'f_nu_grid' : f_nu_y, 'f_sigma_grid' : f_sigma_y }
andersen = mc_price(N_T = 100, absolute_error=5e-2, market_state = state, payoff = payoff, simulate = simulate_heston_andersen_tg, 
                    debug = True,  params = params, T = T, **kwargs)
kwargs = {}
print(andersen)

Number of iterations:   740
Number of simulations:  7400000
Absolute error:         0.0499969627312308
Confidence level:       0.05

26.35216806933968


### Out of the money

In [ ]:
strike = 0.01
payoff = get_payoff(T, strike, state.interest_rate)

### Implied volatility

In [ ]:
def d_1(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r+q*q/2)
    return denom*(log+s2)

def d_2(q,T,S,K,r):
    denom=1/(q*np.sqrt(T))
    log=np.log(S/K)
    s2=T*(r-q*q/2)
    
    return denom*(log+s2)

def calc_iv(option: CallStockOption, state: MarketState, option_price: float):
    
    T=option.expiration_time
    K=option.strike_price
    S=state.stock_price
    r=state.interest_rate    
    N=sps.norm()
    
    
    def f(q):
        d1=d_1(q,T,S,K,r)
        d2=d_2(q,T,S,K,r)
        return S*N.cdf(d1)-K*(np.exp(-r*T))*N.cdf(d2)-option_price
    
    def fprime(q):
        d1=d_1(q,T,S,K,r)
        return S*sps.norm().pdf(d1)*np.sqrt(T)
    
    sol = root_scalar(f, x0=0.5, fprime=fprime, method='newton')
    return sol.root



In [ ]:
strikes = np.arange(80, 120, 2)
strikes

In [ ]:
call_price = np.array([30.096826945107107,
 28.86569299569351,
 27.627749990421822,
 26.468755491540694,
 25.31876428557471,
 24.24782290614315,
 23.16942316862601,
 22.12935496069162,
 21.079816460198398,
 20.20119494244462,
 19.288858679686502,
 18.392107111749084,
 17.50127598061397,
 16.701957005015792,
 15.90122188123928,
 15.14196593075513,
 14.43621999411162,
 13.722786552930495,
 13.069519364876335,
 12.47747576375882])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
model = Heston(state.stock_price, heston_parameters.v0, heston_parameters.kappa, 
                         heston_parameters.vbar, heston_parameters.gamma, heston_parameters.rho, state.interest_rate)

In [ ]:
call_price = np.zeros(20)

for j in range(20):
    call_price[j] = model.call_price(2., strikes[j])

In [ ]:
IV = np.empty_like(call_price)

for j in range(len(call_price)):
        IV[j] = calc_iv(option=CallStockOption(strikes[j], T), 
                           state=state, 
                           option_price=call_price[j])

In [ ]:
_, ax = plt.subplots(figsize=(15, 5))

ax.plot(strikes, IV, "o-")
# ax.legend()
ax.set_xlabel("Strike, $")
ax.set_ylabel("IV")
ax.set_title("Implied Volatility")
plt.show()

In [ ]:
Z = np.random.normal(size=1000)